In [19]:
import pandas as pd
import os
from skimage.transform import resize
from sklearn.utils.validation import column_or_1d
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
import h5py
import re
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.neural_network import MLPRegressor

scorer = make_scorer(mean_squared_error, greater_is_better=False)
param_grid={'C':[0.001],'gamma':[0.1],'kernel':['rbf']}
svr=svm.SVR()
#model = svr
model = MLPRegressor(hidden_layer_sizes=150)
#model = SGDRegressor()
#model=GridSearchCV(svr, param_grid, scoring = scorer)

flat_data_arr=[] #input array
target_arr=[] #output array

#file path
path = 'C:/Users/a0981/OneDrive/桌面/archive/BraTS2020_training_data/content/data'
surInfoFile = "C:/Users/a0981/OneDrive/桌面/archive/BraTS2020_training_data/content/data/survival_info.csv"
MetaDataFile = "C:/Users/a0981/OneDrive/桌面/archive/BraTS2020_training_data/content/data/meta_data.csv"

date_column = ['Brats20ID','Age','Survival_days','Extent_of_Resection']
df_surInfo = pd.read_csv(surInfoFile,names= date_column,skiprows=1)
df_metaData = pd.read_csv(MetaDataFile,names= ['slice_path','target','volume','slice'],skiprows=1)


def modify_value(value):
    idStr = str(value)
    number_str = idStr[-3:] 
    number_int = int(number_str)
    return number_int

#for index,row in df_surInfo.iterrows():
 #   idStr = str(row['Brats20ID'])
  #  number_str = idStr[-3:] 
  #  number_int = int(number_str)
   # surInfoId.append(number_int)
    
df_surInfo['Brats20ID'] = df_surInfo['Brats20ID'].apply(modify_value)
print(df_surInfo)
print(df_metaData)






     Brats20ID     Age  Survival_days Extent_of_Resection
0            1  60.463            289                 GTR
1            2  52.263            616                 GTR
2            3  54.301            464                 GTR
3            4  39.068            788                 GTR
4            5  68.493            465                 GTR
..         ...     ...            ...                 ...
230        363  57.000             62                 GTR
231        366  72.000            633                 GTR
232        367  60.000            437                 STR
233        368  49.000            442                 GTR
234        369  50.000            145                 GTR

[235 rows x 4 columns]
                                  slice_path  target  volume  slice
0         /content/data/volume_41_slice_0.h5       0      41      0
1         /content/data/volume_41_slice_1.h5       0      41      1
2         /content/data/volume_41_slice_2.h5       0      41      2
3       

In [20]:
pattern = r"volume_(\d+)_"
pattern2 = r"slice_(\d+)"

y = pd.DataFrame(columns = ['Survival_days','Volume'])
volumes = []
#path which contains all the categories of images
fileCount = 0
volumeCount = 0
volumeCountMax = 20

volumeId = -1
train_num = int(57195*0.8)
count = 0

MSE = 0.0
dataCount = 0
for img in os.listdir(path):
    
    
    if img.endswith(".h5"):
        count += 1
        
        
        fileName = os.path.join(path,img)
        dataId = re.search(pattern, fileName).group(1)
        dataId = int(dataId)
        SliceId = re.search(pattern2, fileName).group(1)
        SliceId = int(SliceId)
        
        if dataId in df_surInfo['Brats20ID'].values:
            
            
            if volumeId != dataId :
                volumeId = dataId
                df_volume = df_metaData.loc[df_metaData['volume'] == volumeId, ['slice','target']]
                
                volumeCount += 1
                #print(df_volume)
            
            target = df_volume.loc[df_volume['slice'] == SliceId ,'target'].iloc[0]
            
            if volumeCount == volumeCountMax + 1:
            
                flat_data=np.array(flat_data_arr)
                df = pd.DataFrame(flat_data)

                #print(df)
                print(volumes)
                x=df.iloc[:,:-2] #input data 
                Y = df.iloc[:,-2]

                #print(x)
                #print(Y.ravel())
                
                if(count <= train_num):
                
                    print('Splitted Successfully')
                    print(x.shape)
                    print(Y.shape)
                    model.partial_fit(x,Y)
                    print('The Model is trained well with the given images')
                
                else:
                    y_pred=model.predict(x)
                    print("The predicted Data is :")
                    print(y_pred)
                    print("The actual data is:")
                    print(np.array(Y))
                    mse = mean_squared_error(y_pred,Y)
                    MSE += mse*len(Y)
                    dataCount += len(Y)
                    print(f"The model MSE is {mse}")

                flat_data_arr = []
                y = pd.DataFrame(columns = ['Survival_days'])
                fileCount = 0
                volumeCount = 1
                volumes = []
            
            if target == 1:
                
                #print(f'dataId :{dataId}')
                #print(f'SliceId :{SliceId}')
                f = h5py.File(fileName, 'r')
                img_array = f.get('image')
                img_array = np.array(img_array)
                img_array = img_array.flatten()
                
                if dataId not in volumes:
                    volumes.append(dataId)
            
                age = df_surInfo[df_surInfo['Brats20ID']==dataId]['Age'].values[0]
                img_array = np.append(img_array,age)
                survDays = df_surInfo.loc[df_surInfo['Brats20ID'] == dataId, 'Survival_days'].iloc[0]
                img_array = np.append(img_array,survDays)
                img_array = np.append(img_array,dataId)
            
                flat_data_arr.append(img_array)
                fileCount += 1
            
print(f"dataCount : {dataCount}")
print(f"The model MSE is {MSE/dataCount}")


[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 10, 110, 111, 112, 113, 114, 115, 116, 117, 118]
Splitted Successfully
(1278, 230401)
(1278,)
The Model is trained well with the given images
[119, 11, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 12, 13, 145, 147, 14, 151, 152, 153]
Splitted Successfully
(1301, 230401)
(1301,)
The Model is trained well with the given images
[154, 157, 158, 159, 15, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 16, 170, 173, 174, 175]
Splitted Successfully
(1396, 230401)
(1396,)
The Model is trained well with the given images
[177, 178, 179, 17, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 18, 190, 191, 192, 194, 195]
Splitted Successfully
(1366, 230401)
(1366,)
The Model is trained well with the given images
[196, 199, 19, 1, 201, 203, 204, 206, 20, 210, 214, 215, 216, 217, 218, 219, 21, 220, 221, 222]
Splitted Successfully
(1266, 230401)
(1266,)
The Model is trained well with the given images
[223, 224, 225, 226, 227, 228, 229, 22, 230,